# 0. Install Dependencies

In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym 
import random
import os


In [2]:
from pyglet import *
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

/Users/casa/miniconda3/envs/invaders/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

2025-10-26 17:53:28.790 python[1929:52808] isPrefsCreateCacheFromEnabledAndDefaultInputSources - can't find anything from GetInputSourceEnabledPrefs, use defaultASCIIKeyLayoutDict = <CFBasicHash 0x7fe41349cad0 [0x7fff9c2e48e0]>{type = mutable dict, count = 3,
entries =>
	0 : <CFString 0x7fff9c34f7d8 [0x7fff9c2e48e0]>{contents = "InputSourceKind"} = <CFString 0x7fff9c394d18 [0x7fff9c2e48e0]>{contents = "Keyboard Layout"}
	1 : <CFString 0x7fff9c380e58 [0x7fff9c2e48e0]>{contents = "KeyboardLayout ID"} = <CFNumber 0x5737 [0x7fff9c2e48e0]>{value = +87, type = kCFNumberSInt64Type}
	9 : <CFString 0x7fff9c34a4d8 [0x7fff9c2e48e0]>{contents = "KeyboardLayout Name"} = <CFString 0x7fff9c3d2d38 [0x7fff9c2e48e0]>{contents = "Spanish - ISO"}
}


Episode:1 Score:95.0
Episode:2 Score:80.0
Episode:3 Score:155.0
Episode:4 Score:230.0
Episode:5 Score:210.0


# 2. Create a Deep Learning Model with Keras

In [29]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [30]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
del model

In [31]:
model = build_model(height, width, channels, actions)

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 6)                

# 3. Build Agent with Keras-RL

In [33]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [34]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [35]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [18]:
dqn.fit(env, nb_steps=2000, visualize=False, verbose=2)

Training for 2000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  702/2000: episode: 1, duration: 18.961s, episode steps: 702, steps per second:  37, episode reward: 190.000, mean reward:  0.271 [ 0.000, 30.000], mean action: 2.519 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1090/2000: episode: 2, duration: 177.338s, episode steps: 388, steps per second:   2, episode reward: 35.000, mean reward:  0.090 [ 0.000, 15.000], mean action: 2.420 [0.000, 5.000],  loss: 32.137455, mean_q: 12.496752, mean_eps: 0.905950
 1659/2000: episode: 3, duration: 909.179s, episode steps: 569, steps per second:   1, episode reward: 140.000, mean reward:  0.246 [ 0.000, 30.000], mean action: 2.594 [0.000, 5.000],  loss: 1.225459, mean_q: 10.759120, mean_eps: 0.876340
done, took 1618.734 seconds


In [45]:
scores = dqn.test(env, nb_episodes=2, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 2 episodes ...
Episode 1: reward: 165.000, steps: 637
Episode 2: reward: 285.000, steps: 973
225.0


# 4. Reloading Agent from Memory

In [20]:
dqn.save_weights('SavedWeights/2k-Fast/dqn_weights.h5f')

In [21]:
del model, dqn

In [41]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')